In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import pandas as pd
from colorama import Fore
import pickle as pkl
from tqdm import tqdm
import data, viz
from style import *
out_dir = 'results/jul22_11' # jun17_1

# compare all models

In [ ]:
def load_results(out_dir):
    r = []
    for fname in tqdm(os.listdir(out_dir)):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        d['param_name'] = fname[:-4]
        r.append(pd.Series(d))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('param_name')
    return r

results = load_results(out_dir)
# print(results.keys())

  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
results.head()

## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [ ]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
# r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
# r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)

# filter out if it always predits positive
r = r[r['tn_cv'] > 0]

r = r.sort_values(by=['sensitivity_cv', 'specificity_cv'], ascending=False)
# r = r[['fn', 'tn', 'fp', 'tp', 'specificity', 'sensitivity', 'roc_auc', 'precision',
#        'f1', 'balanced_accuracy', 'accuracy']]
r = r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2',
       'fn_cv', 'tn_cv', 'fp_cv', 'tp_cv',
       'precision_cv', 'roc_auc_cv', 
       'f1_cv', 'balanced_accuracy_cv', 'accuracy_cv',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
# r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

In [ ]:
r

**look at feat importances**

In [23]:
r = results
best_model = 'logistic_select_rf=10_sample_weights=1000' #logistic_select_rf=5_smote=5'
# print(r.keys())
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r.sort_values('balanced_accuracy', ascending=False)
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('35')]
# r = r[r.index.str.contains('11')]
# r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('20')]
r = r[r.index.str.contains(best_model)]

r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
# r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank
idxs = r.index
# r.insert(0, 'balanced_acc', results.loc[idxs]['balanced_accuracy'])
# r = r.sort_values('balanced_acc', ascending=False)
# subset = list(r.keys())
# subset.remove('balanced_acc')
# r.to_html('table.html')
# r = r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1) #, subset=subset) # rows differently colored


r = r.transpose()
r = r.reindex(r[best_model].abs().sort_values(ascending=False).index) # sort by abs value
# r = r.sort_values(by='logistic_20_ros=1')
r = r.fillna(0).style.background_gradient(cmap=cm) #, axis=0) #, subset=subset) # rows differently colored
# r = background_gradient(r)
r

model_type,logistic_select_rf=10_sample_weights=1000
RtCostalTender,1.079
AbdTrauma_or_SeatBeltSign_yes,0.825
AbdTenderDegree_Severe,0.631
InitHeartRate,0.391
AbdDistention_yes,0.389
InitSysBPRange,-0.323
VomitWretch_yes,0.272
Age,0.197
MOI_Motor vehicle collision,0.197
ThoracicTrauma_yes,-0.017


# misc analysis

In [ ]:
def count_common_feats(m_cv, model_type='tree'):
    if model_type == 'tree':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            feats_used += list(ks[m_fit.feature_importances_ != 0])
    elif model_type == 'logistic':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            num_feats = 5
            # get top num_feats features with biggest bsolute weights
            idxs = np.abs(m_fit.coef_).flatten().argsort()[-num_feats:][::-1]
            feats_used += list(ks[idxs])
    return sorted(dict(Counter(feats_used)).items(), key=lambda kv: kv[1], reverse=True)
        
count_common_feats(m_cv, model_type)

In [ ]:
# plot a tree
m_fit = m_cv['estimator'][0]
plt.figure(dpi=300)
plot_tree(m_fit, feature_names=ks)
plt.show()